In [36]:
import pandas as pd

from collections import Counter
from datetime import datetime

In [37]:
filename = "melon_chart.csv"
raw_data = pd.read_csv(filename)

In [38]:
raw_data.head()


,rank,title,singer,album,date,like
0,1,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night,2024.01.01.01,114191
1,2,Drama,aespa,Drama - The 4th Mini Album,2024.01.01.01,85818
2,3,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,2024.01.01.01,143931
3,4,비의 랩소디,임재현,비의 랩소디,2024.01.01.01,68931
4,5,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',2024.01.01.01,287040


In [39]:
# ----------------------------------------------------------#
# # 해당 부분은 현재 해결하여 신규 데이터부터 적용할 필요없음.
# raw_data['tmp'] = raw_data['album']
# raw_data['album'] = raw_data['singer']
# raw_data['singer'] = raw_data['tmp']
# raw_data.drop(['tmp'],axis=1,inplace = True)
# raw_data.head()
# ----------------------------------------------------------#

# 1. 봄 / 겨울 구분 컬럼 추가
# 봄 : 0 겨울 : 1 
data = raw_data
data[['year','month','day','week']] = data['date'].str.split('.',expand=True)
data['year'] = data['year'].astype('int')
data['month'] = data['month'].astype('int')
data['day'] = data['day'].astype('int')
data['week'] = data['week'].astype('int')
data['season'] = data['month'].apply(lambda x : 1 if x in [1,2,11,12] else 0) # 1 : 겨울 / 0 : 봄 

data.head()

,rank,title,singer,album,date,like,year,month,day,week,season
0,1,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night,2024.01.01.01,114191,2024,1,1,1,1
1,2,Drama,aespa,Drama - The 4th Mini Album,2024.01.01.01,85818,2024,1,1,1,1
2,3,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,2024.01.01.01,143931,2024,1,1,1,1
3,4,비의 랩소디,임재현,비의 랩소디,2024.01.01.01,68931,2024,1,1,1,1
4,5,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',2024.01.01.01,287040,2024,1,1,1,1


In [40]:
data.shape

(37854, 11)

# 0. 동일년도 봄에도 등장하고, 겨울에도 등장하는 노래 조회.

In [41]:
spring_songs = data[data['month'].isin([3,4,5])]
winter_songs = data[data['month'].isin([1,2,12])]

In [42]:
print(spring_songs.shape,winter_songs.shape)

(18174, 11) (18980, 11)


In [43]:
# 동일한 년도에 등장한 노래
both_table = pd.merge(spring_songs,winter_songs,on=['title','year'])

In [44]:
both_table

,rank_x,title,singer_x,album_x,date_x,like_x,year,month_x,day_x,week_x,season_x,rank_y,singer_y,album_y,date_y,like_y,month_y,day_y,week_y,season_y
0,1,밤양갱,비비 (BIBI),밤양갱,2024.03.04.01,133504,2024,3,4,1,0,21,비비 (BIBI),밤양갱,2024.02.12.03,133503,2,12,3,1
1,1,밤양갱,비비 (BIBI),밤양갱,2024.03.04.01,133504,2024,3,4,1,0,2,비비 (BIBI),밤양갱,2024.02.19.04,133503,2,19,4,1
2,1,밤양갱,비비 (BIBI),밤양갱,2024.03.04.01,133504,2024,3,4,1,0,1,비비 (BIBI),밤양갱,2024.02.26.05,133503,2,26,5,1
3,2,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,2024.03.04.01,114368,2024,3,4,1,0,61,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,2024.01.29.01,114367,1,29,1,1
4,2,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,2024.03.04.01,114368,2024,3,4,1,0,11,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,2024.02.05.02,114367,2,5,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77870,94,죽어도 못 보내,2am,죽어도 못 보내,2010.05.23.04,27194,2010,5,23,4,0,2,2am,죽어도 못 보내,2010.01.31.01,27194,1,31,1,1
77871,94,죽어도 못 보내,2am,죽어도 못 보내,2010.05.23.04,27194,2010,5,23,4,0,1,2am,죽어도 못 보내,2010.02.07.02,27194,2,7,2,1
77872,94,죽어도 못 보내,2am,죽어도 못 보내,2010.05.23.04,27194,2010,5,23,4,0,1,2am,죽어도 못 보내,2010.02.14.03,27194,2,14,3,1
77873,94,죽어도 못 보내,2am,죽어도 못 보내,2010.05.23.04,27194,2010,5,23,4,0,2,2am,죽어도 못 보내,2010.02.21.04,27194,2,21,4,1


In [45]:
# 원본데이터에서 동일한 년도에 등장한 노래 제거
joined_data = pd.merge(data, both_table, left_on=['title', 'singer','year'], right_on=['title', 'singer_x','year'], how='left')
result_data = joined_data[pd.isna(joined_data['singer_x'])] # 동일한 년도에 등장하지 않았다면 singer_x 가 nan 일 것 이기 떄문에. 

In [46]:
result_data.shape

(17205, 29)

In [47]:
result_data.head()

,rank,title,singer,album,date,like,year,month,day,week,...,season_x,rank_y,singer_y,album_y,date_y,like_y,month_y,day_y,week_y,season_y
2115,43,잠시라도 우리,"성시경, 나얼",잠시라도 우리,2024.01.01.01,39656,2024,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2422,50,Chill Kill,Red Velvet (레드벨벳),Chill Kill - The 3rd Album,2024.01.01.01,42790,2024,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2666,56,화이트 (White),폴킴,화이트 (White),2024.01.01.01,18814,2024,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2667,57,눈이 오잖아(Feat.헤이즈),이무진,눈이 오잖아(Feat.헤이즈),2024.01.01.01,135142,2024,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2776,60,GODS,"NewJeans, League of Legends",2023 리그 오브 레전드 월드 챔피언십 주제곡,2024.01.01.01,46175,2024,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# 필요한 행만 추출 
data.columns
result_data = result_data[['rank', 'title', 'singer', 'album', 'date', 'like', 'year', 'month',
       'day', 'week', 'season']]

In [49]:
result_data

,rank,title,singer,album,date,like,year,month,day,week,season
2115,43,잠시라도 우리,"성시경, 나얼",잠시라도 우리,2024.01.01.01,39656,2024,1,1,1,1
2422,50,Chill Kill,Red Velvet (레드벨벳),Chill Kill - The 3rd Album,2024.01.01.01,42790,2024,1,1,1,1
2666,56,화이트 (White),폴킴,화이트 (White),2024.01.01.01,18814,2024,1,1,1,1
2667,57,눈이 오잖아(Feat.헤이즈),이무진,눈이 오잖아(Feat.헤이즈),2024.01.01.01,135142,2024,1,1,1,1
2776,60,GODS,"NewJeans, League of Legends",2023 리그 오브 레전드 월드 챔피언십 주제곡,2024.01.01.01,46175,2024,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1512393,96,그날 이후로,나윤권,김형석 With 나윤권,2010.12.26.05,2379,2010,12,26,5,1
1512394,97,I'll Be Back,2PM,Still 2:00pm,2010.12.26.05,9493,2010,12,26,5,1
1512395,98,널 지우는 일,김지수,널 지우는 일,2010.12.26.05,1635,2010,12,26,5,1
1512396,99,Thanks To,"용준형, 양요섭",My Story,2010.12.26.05,18397,2010,12,26,5,1


In [50]:
result_data.to_csv("./except_both_title.csv",index=False)

In [51]:
result_data.loc[result_data.title == '봄봄봄']

,rank,title,singer,album,date,like,year,month,day,week,season
1268720,86,봄봄봄,로이킴,봄봄봄,2015.03.23.04,137501,2015,3,23,4,0
1272629,87,봄봄봄,로이킴,봄봄봄,2015.03.30.01,137501,2015,3,30,1,0
1276416,90,봄봄봄,로이킴,봄봄봄,2015.04.06.02,137501,2015,4,6,2,0
1394843,1,봄봄봄,로이킴,봄봄봄,2013.04.22.04,137502,2013,4,22,4,0
1396350,1,봄봄봄,로이킴,봄봄봄,2013.04.29.01,137502,2013,4,29,1,0
1397812,2,봄봄봄,로이킴,봄봄봄,2013.05.06.02,137502,2013,5,6,2,0
1399158,2,봄봄봄,로이킴,봄봄봄,2013.05.13.03,137502,2013,5,13,3,0
1400486,3,봄봄봄,로이킴,봄봄봄,2013.05.20.04,137502,2013,5,20,4,0
1401730,5,봄봄봄,로이킴,봄봄봄,2013.05.27.05,137502,2013,5,27,5,0


In [52]:
result_data.loc[result_data.title=='벚꽃 엔딩']

,rank,title,singer,album,date,like,year,month,day,week,season
58429,52,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2024.04.01.01,345227,2024,4,1,1,0
62357,49,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2024.04.08.02,345227,2024,4,8,2,0
165076,53,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2023.03.27.05,345227,2023,3,27,5,0
174112,67,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2023.04.03.01,345227,2023,4,3,1,0
183998,96,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2023.04.10.02,345227,2023,4,10,2,0
345004,99,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2022.03.28.05,345227,2022,3,28,5,0
348518,43,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2022.04.04.01,345227,2022,4,4,1,0
356611,51,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2022.04.11.02,345227,2022,4,11,2,0
367165,82,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2022.04.18.03,345227,2022,4,18,3,0
375102,100,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2022.04.25.04,345227,2022,4,25,4,0


In [53]:
data.loc[(data.title == '벚꽃 엔딩')&(data.season == 1)]

,rank,title,singer,album,date,like,year,month,day,week,season
17986,87,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2017.02.27.01,345228,2017,2,27,1,1
20471,75,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2016.02.29.01,345228,2016,2,29,1,1
23091,95,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2015.02.23.04,345230,2015,2,23,4,1
25789,93,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2014.02.24.04,345231,2014,2,24,4,1
28382,87,벚꽃 엔딩,버스커 버스커,버스커 버스커 1집,2013.02.25.04,345231,2013,2,25,4,1


In [54]:
data.loc[data.title=='Thanks To']

,rank,title,singer,album,date,like,year,month,day,week,season
37852,99,Thanks To,"용준형, 양요섭",My Story,2010.12.26.05,18397,2010,12,26,5,1


In [55]:
data.loc[data.title == '화이트 (White)']

,rank,title,singer,album,date,like,year,month,day,week,season
55,56,화이트 (White),폴킴,화이트 (White),2024.01.01.01,18814,2024,1,1,1,1
179,80,화이트 (White),폴킴,화이트 (White),2024.01.08.02,18814,2024,1,8,2,1
282,83,화이트 (White),폴킴,화이트 (White),2024.01.15.03,18814,2024,1,15,3,1
392,93,화이트 (White),폴킴,화이트 (White),2024.01.22.04,18814,2024,1,22,4,1
3888,89,화이트 (White),폴킴,화이트 (White),2023.12.18.03,18814,2023,12,18,3,1
3932,33,화이트 (White),폴킴,화이트 (White),2023.12.25.04,18814,2023,12,25,4,1


# 1. 연도-계절 별 차트 등장회수 TOP5 조회

In [56]:
data = pd.read_csv("./except_both_title.csv")
data.head()

,rank,title,singer,album,date,like,year,month,day,week,season
0,43,잠시라도 우리,"성시경, 나얼",잠시라도 우리,2024.01.01.01,39656,2024,1,1,1,1
1,50,Chill Kill,Red Velvet (레드벨벳),Chill Kill - The 3rd Album,2024.01.01.01,42790,2024,1,1,1,1
2,56,화이트 (White),폴킴,화이트 (White),2024.01.01.01,18814,2024,1,1,1,1
3,57,눈이 오잖아(Feat.헤이즈),이무진,눈이 오잖아(Feat.헤이즈),2024.01.01.01,135142,2024,1,1,1,1
4,60,GODS,"NewJeans, League of Legends",2023 리그 오브 레전드 월드 챔피언십 주제곡,2024.01.01.01,46175,2024,1,1,1,1


In [57]:
# 연도별 계절에 따른 가장 많이 등장한 title,singer,date 
# date ???

# 결과를 저장할 빈 데이터 프레임을 생성
columns=['year', 'season','title', 'singer', 'count']
result_df = pd.DataFrame(columns=columns)

temp = []

# grouped_data = data.groupby(['year','season','title','singer']).size().reset_index(name='count')
grouped_data = data.groupby(['year','season'])

for name,group in grouped_data:
    year,season = name

    # 노래 제목과 가수로 튜플로 만들어 등장회수 세기.
    song_counter = Counter(zip(group['title'],group['singer'])) # {(노래1,가수1):2,(노래2,가수1):1, ....}
    most_song = song_counter.most_common(5) # 상위 5개 [((노래1,가수1),2).... ]

    for (title,singer),count in most_song:
        temp_df = pd.DataFrame(
            {
                'year' : [year],
                'season' : [season],
                'title' : [title],
                'singer' : [singer],
                'count' : [count]
            }
        )
        temp.append(temp_df)
result_df = pd.concat(temp,ignore_index=True)
        



In [58]:
result_df

,year,season,title,singer,count
0,2010,0,선물 (Feat. 은지원),케이윌,12
1,2010,0,사랑이 올까요 (With 백지영),"마이티 마우스, 백지영",12
2,2010,0,미워요,정인,12
3,2010,0,잘못했어,2am,11
4,2010,0,Run Devil Run,소녀시대 (GIRLS' GENERATION),11
...,...,...,...,...,...
145,2024,1,잠시라도 우리,"성시경, 나얼",9
146,2024,1,눈이 오잖아(Feat.헤이즈),이무진,9
147,2024,1,Chill Kill,Red Velvet (레드벨벳),8
148,2024,1,GODS,"NewJeans, League of Legends",7


# 2. 3년 이상 연속으로 등장한 노래 추출

In [59]:
data.head()

,rank,title,singer,album,date,like,year,month,day,week,season
0,43,잠시라도 우리,"성시경, 나얼",잠시라도 우리,2024.01.01.01,39656,2024,1,1,1,1
1,50,Chill Kill,Red Velvet (레드벨벳),Chill Kill - The 3rd Album,2024.01.01.01,42790,2024,1,1,1,1
2,56,화이트 (White),폴킴,화이트 (White),2024.01.01.01,18814,2024,1,1,1,1
3,57,눈이 오잖아(Feat.헤이즈),이무진,눈이 오잖아(Feat.헤이즈),2024.01.01.01,135142,2024,1,1,1,1
4,60,GODS,"NewJeans, League of Legends",2023 리그 오브 레전드 월드 챔피언십 주제곡,2024.01.01.01,46175,2024,1,1,1,1


In [60]:
def is_consecutive(years_set):
    sorted_years = sorted(years_set) # 연도 오름차순 정렬
    consecutive_count = 1 # 최소 1년은 등장하므로 1로 초기화.
    max_consecutive = 1 # 연속연도 count 변수

    for i in range(len(sorted_years)-1):
        # 현재연도와 다음연도가 연속적인지 확인. 2023+1 == 2024 
        if sorted_years[i] + 1 == sorted_years[i + 1] :
            consecutive_count += 1
            max_consecutive = max(max_consecutive,consecutive_count)
        else : 
            consecutive_count = 1
    return max_consecutive >= 3 # 3년연속 등장여부

In [61]:
# 연도를 기준으로 등장 연도 조회 ( 중복제거를 위해 set 사용 )
grouped_data = data.groupby(['title','singer'])['year'].apply(set).reset_index()

In [62]:
# 3년이상 연속출현 여부 , 1 : 출현 0 : 미출현
grouped_data['more_than_3'] = grouped_data['year'].apply(lambda x : 1 if is_consecutive(x) else 0 )

In [63]:
# top100 차트인 했던 연도수 리스트컬럼 
grouped_data['year'] = grouped_data['year'].apply(lambda x :list(x))

In [64]:
result_df_2 = grouped_data.rename(columns = {'year':'years'})

In [65]:
result_df_2.loc[result_df_2.more_than_3 == 1]

,title,singer,years,more_than_3
87,All I Want for Christmas Is You,Mariah Carey,"[2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...",1
464,Gone,브라운 아이드 소울,"[2010, 2011, 2012]",1
674,Last Christmas,Ariana Grande,"[2018, 2020, 2021, 2022, 2023]",1
795,Make It To Christmas,Alessia Cara,"[2021, 2022, 2023]",1
827,Must Have Love,"SG 워너비, 브라운아이드걸스","[2018, 2019, 2021, 2022, 2023, 2014]",1
1033,Santa Tell Me,Ariana Grande,"[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]",1
1081,Snowman,Sia,"[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]",1
1162,Text Me Merry Christmas (Feat. Kristen Bell),Straight No Chaser,"[2019, 2020, 2021, 2022, 2023]",1
1232,Underneath The Tree,Kelly Clarkson,"[2021, 2022, 2023]",1
1746,꽃송이가,버스커 버스커,"[2016, 2012, 2013, 2014, 2015]",1


# 3-1. Add : 시즌에 따라 테이블 분류 (봄테이블, 겨울테이블)

In [66]:
# 봄에 차트인 했던 노래 가운데 3년 연속 등장했던 노래 목록
spring_season = data.loc[data.season == 0 ]
spring_grouped = spring_season.groupby(['title','singer'])['year'].apply(set).reset_index()
spring_grouped['more_than_3'] = spring_grouped['year'].apply(lambda x : 1 if is_consecutive(x) else 0 )
spring_grouped['year'] = spring_grouped['year'].apply(lambda x :list(x))
spring_grouped_rs = spring_grouped.rename(columns = {'year':'years'})
spring_grouped_rs

,title,singer,years,more_than_3
0,#결별,"길구봉구, 박보람",[2018],0
1,...하고 싶다,V.One,[2010],0
2,..IS YOU,XIA (준수),[2016],0
3,0330,유키스,[2011],0
4,100 Percent,이효리,[2010],0
...,...,...,...,...
1895,흩어져,먼데이 키즈 (Monday Kiz),[2010],0
1896,희나리,최준영,[2012],0
1897,희야,하예나,[2012],0
1898,희재,이수영,[2013],0


In [67]:
spring_grouped_rs.loc[spring_grouped_rs.more_than_3 == 1 ]

,title,singer,years,more_than_3
800,꽃송이가,버스커 버스커,"[2016, 2012, 2013, 2014, 2015]",1
1190,벚꽃 엔딩,버스커 버스커,"[2018, 2019, 2020, 2021, 2022, 2023, 2024, 2012]",1
1216,봄 사랑 벚꽃 말고,"HIGH4 (하이포), 아이유","[2016, 2017, 2018, 2020, 2021, 2022, 2023, 202...",1
1231,봄이 좋냐??,10CM,"[2016, 2017, 2018]",1
1621,우연히 봄,"로꼬, 유주 (YUJU)","[2016, 2020, 2022, 2023, 2024, 2015]",1


In [71]:
# 겨울에 차트인 했던 노래 가운데 3년 연속 등장했던 노래 목록
winter_season = data.loc[data.season == 1 ]
winter_grouped = winter_season.groupby(['title','singer'])['year'].apply(set).reset_index()
winter_grouped['more_than_3'] = winter_grouped['year'].apply(lambda x : 1 if is_consecutive(x) else 0 )
winter_grouped['year'] = winter_grouped['year'].apply(lambda x :list(x))
winter_grouped_rs = winter_grouped.rename(columns = {'year':'years'})
winter_grouped_rs

,title,singer,years,more_than_3
0,0310,백예린 (Yerin Baek),[2019],0
1,"08베이식 (Feat. 염따, punchnello)",베이식 (Basick),[2021],0
2,"1,2,3,4 (원,투,쓰리,포)",이하이,[2012],0
3,11:11,태연 (TAEYEON),[2016],0
4,12:45 (Stripped),Etham,[2020],0
...,...,...,...,...
2385,히치하이킹 (Hitchhiking),SHINee (샤이니),[2013],0
2386,힐링이 필요해,로이킴,"[2012, 2013]",0
2387,힘든 건 사랑이 아니다,임창정,[2020],0
2388,"힘을 내요, 그대",더필름,[2011],0


In [72]:
winter_grouped_rs.loc[winter_grouped_rs.more_than_3 == 1 ]

,title,singer,years,more_than_3
43,All I Want for Christmas Is You,Mariah Carey,"[2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...",1
243,Gone,브라운 아이드 소울,"[2010, 2011, 2012]",1
363,Last Christmas,Ariana Grande,"[2018, 2020, 2021, 2022, 2023]",1
424,Make It To Christmas,Alessia Cara,"[2021, 2022, 2023]",1
441,Must Have Love,"SG 워너비, 브라운아이드걸스","[2018, 2019, 2021, 2022, 2023, 2014]",1
557,Santa Tell Me,Ariana Grande,"[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]",1
580,Snowman,Sia,"[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]",1
627,Text Me Merry Christmas (Feat. Kristen Bell),Straight No Chaser,"[2019, 2020, 2021, 2022, 2023]",1
664,Underneath The Tree,Kelly Clarkson,"[2021, 2022, 2023]",1
1419,미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ),아이유,"[2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...",1


In [75]:
spring_grouped_rs.loc[spring_grouped_rs.more_than_3 == 1 ].to_csv("./csv_file/spring_more_than3.csv",index=False)
winter_grouped_rs.loc[winter_grouped_rs.more_than_3 == 1 ].to_csv("./csv_file/winter_more_than3.csv",index=False)



# 3. 각 노래별 최초 등장일 조회

In [17]:
data.head()

,rank,title,singer,album,date,like,month,year,season
0,1,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night,2024.01.01 ~ 2024.01.07,114126,1,2024,1
1,2,Drama,aespa,Drama - The 4th Mini Album,2024.01.01 ~ 2024.01.07,85715,1,2024,1
2,3,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,2024.01.01 ~ 2024.01.07,143705,1,2024,1
3,4,비의 랩소디,임재현,비의 랩소디,2024.01.01 ~ 2024.01.07,68730,1,2024,1
4,5,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',2024.01.01 ~ 2024.01.07,287039,1,2024,1


In [18]:
data = raw_data
data['start_date'] = data['date'].apply(lambda x : x.split('~')[0].strip())
data['start_date'] = pd.to_datetime(data['start_date'])

result_df_3 = data.groupby(['title','singer'])['start_date'].min().reset_index()

In [19]:
result_df_3

,title,singer,start_date
0,#결별,"길구봉구, 박보람",2018-04-16
1,#첫사랑,볼빨간사춘기,2018-01-08
2,%% (응응),Apink (에이핑크),2019-01-07
3,...하고 싶다,V.One,2010-03-07
4,..IS YOU,XIA (준수),2016-05-16
...,...,...,...
4961,히히하헤호,"마마무 (Mamamoo), 긱스 (Geeks)",2014-05-26
4962,힐링이 필요해,로이킴,2012-12-03
4963,힘든 건 사랑이 아니다,임창정,2020-11-30
4964,"힘을 내요, 그대",더필름,2011-01-02


# 4. 3년 이상 등장한 음원 목록 조회.

In [20]:
data = raw_data
data.head()

,rank,title,singer,album,date,like,month,year,season,start_date
0,1,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night,2024.01.01 ~ 2024.01.07,114126,1,2024,1,2024-01-01
1,2,Drama,aespa,Drama - The 4th Mini Album,2024.01.01 ~ 2024.01.07,85715,1,2024,1,2024-01-01
2,3,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,2024.01.01 ~ 2024.01.07,143705,1,2024,1,2024-01-01
3,4,비의 랩소디,임재현,비의 랩소디,2024.01.01 ~ 2024.01.07,68730,1,2024,1,2024-01-01
4,5,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',2024.01.01 ~ 2024.01.07,287039,1,2024,1,2024-01-01


In [21]:
# 중복 연도수 제거
grouped_data = data.groupby(['title','singer'])['year'].apply(set).reset_index()
grouped_data['year'] = grouped_data['year'].apply(lambda x : list(x))
grouped_data.head()

,title,singer,year
0,#결별,"길구봉구, 박보람",[2018]
1,#첫사랑,볼빨간사춘기,[2018]
2,%% (응응),Apink (에이핑크),[2019]
3,...하고 싶다,V.One,[2010]
4,..IS YOU,XIA (준수),[2016]


In [28]:
# 등장년도 회수 칼럼
grouped_data['chartin_counts'] = grouped_data['year'].apply(lambda x : len(x))

# 등장년도 3이상 음원 조회
result_df_4 = grouped_data[grouped_data['chartin_counts'] >= 3]



,title,singer,years,chartin_counts
38,2002,Anne-Marie,"[2019, 2020, 2021]",3
69,A,Ariana Grande,"[2018, 2014, 2015]",3
79,A,Maroon 5,"[2016, 2017, 2018, 2011, 2012, 2014, 2015]",7
108,A,이해리 (다비치),"[2012, 2013, 2014]",3
115,A bientot,임영웅,"[2024, 2022, 2023]",3
...,...,...,...,...
4696,크리스마스니까,"성시경, 박효신, 이석훈, 서인국, VIXX (빅스)","[2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...",13
4753,피 땀 눈물,방탄소년단,"[2016, 2017, 2018]",3
4820,한숨,이하이,"[2016, 2017, 2018]",3
4838,해요 (2022),#안녕,"[2024, 2022, 2023]",3


# 5. 3년 이상 등장한 데이터 + 3년 연속등장한 데이터 JOIN

In [29]:
result_df_2.head()

,title,singer,years,more_than_3
0,#결별,"길구봉구, 박보람",[2018],0
1,#첫사랑,볼빨간사춘기,[2018],0
2,%% (응응),Apink (에이핑크),[2019],0
3,...하고 싶다,V.One,[2010],0
4,..IS YOU,XIA (준수),[2016],0


In [32]:
print(result_df_2.shape,result_df_4.shape)

(4966, 4) (105, 4)


In [33]:
tmp_result_df_4 = grouped_data

tmp_result_df_4 = tmp_result_df_4.rename(columns={'year':'years'})

In [34]:
print(result_df_2.shape,tmp_result_df_4.shape)

(4966, 4) (4966, 4)


In [37]:
result_df_5 = pd.merge(left=result_df_2,right=tmp_result_df_4,on=['title','singer'])

In [39]:
result_df_5.columns

Index(['title', 'singer', 'years_x', 'more_than_3', 'years_y',
       'chartin_counts'],
      dtype='object')

In [40]:
result_df_5.drop(['years_x'],axis=1,inplace=True)

In [42]:
result_df_5 = result_df_5.rename(columns={'years_y':'years'})

In [43]:
result_df_5

,title,singer,more_than_3,years,chartin_counts
0,#결별,"길구봉구, 박보람",0,[2018],1
1,#첫사랑,볼빨간사춘기,0,[2018],1
2,%% (응응),Apink (에이핑크),0,[2019],1
3,...하고 싶다,V.One,0,[2010],1
4,..IS YOU,XIA (준수),0,[2016],1
...,...,...,...,...,...
4961,히히하헤호,"마마무 (Mamamoo), 긱스 (Geeks)",0,[2014],1
4962,힐링이 필요해,로이킴,0,"[2012, 2013]",2
4963,힘든 건 사랑이 아니다,임창정,0,"[2020, 2021]",2
4964,"힘을 내요, 그대",더필름,0,[2011],1


In [44]:
# 3년 이상 등장 데이터 조회
result_df_5.loc[result_df_5.chartin_counts >= 3]

,title,singer,more_than_3,years,chartin_counts
38,2002,Anne-Marie,1,"[2019, 2020, 2021]",3
69,A,Ariana Grande,0,"[2018, 2014, 2015]",3
79,A,Maroon 5,1,"[2016, 2017, 2018, 2011, 2012, 2014, 2015]",7
108,A,이해리 (다비치),1,"[2012, 2013, 2014]",3
115,A bientot,임영웅,1,"[2024, 2022, 2023]",3
...,...,...,...,...,...
4696,크리스마스니까,"성시경, 박효신, 이석훈, 서인국, VIXX (빅스)",1,"[2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...",13
4753,피 땀 눈물,방탄소년단,1,"[2016, 2017, 2018]",3
4820,한숨,이하이,1,"[2016, 2017, 2018]",3
4838,해요 (2022),#안녕,1,"[2024, 2022, 2023]",3


In [45]:
# 3년 연속 등장 데이터 조회
result_df_5.loc[result_df_5.more_than_3 == 1]

,title,singer,more_than_3,years,chartin_counts
38,2002,Anne-Marie,1,"[2019, 2020, 2021]",3
79,A,Maroon 5,1,"[2016, 2017, 2018, 2011, 2012, 2014, 2015]",7
108,A,이해리 (다비치),1,"[2012, 2013, 2014]",3
115,A bientot,임영웅,1,"[2024, 2022, 2023]",3
131,After LIKE,IVE (아이브),1,"[2024, 2022, 2023]",3
...,...,...,...,...,...
4696,크리스마스니까,"성시경, 박효신, 이석훈, 서인국, VIXX (빅스)",1,"[2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...",13
4753,피 땀 눈물,방탄소년단,1,"[2016, 2017, 2018]",3
4820,한숨,이하이,1,"[2016, 2017, 2018]",3
4838,해요 (2022),#안녕,1,"[2024, 2022, 2023]",3
